In [7]:
# mkdir dataset
# cd dataset
# wget --no-check-certificate https://www.statmt.org/europarl/v7/es-en.tgz
# tar -zxvf es-en.tgz
# These files names do not exactly match the ones below, so maybe wrong file


In [8]:
import pandas as pd
import numpy as np
import re
from collections import Counter

# chose a random value to get the code to work
MAX_VOCABULARY = 10

# EuroParl English-Spanish Dataset

In this notebook, I explore the EuroParl dataset for English/Spanish translation. My goal is to gain a somewhat good understanding of the data.

## Loading the Data
First, we load the datasets. There are two files provided when we download the Europarl dataset: `Europarl.<lan_1>-<lan-2>.<lan_1>` and `Europarl.<lan_1>-<lan-2>.<lan_2>` It is up to the user to decide which one is the data and which one are the labels.

In [9]:
# europarl_english_path = "dataset/Europarl.en-es.en"
europarl_english_path = "dataset/europarl-v7.es-en.en"

# Load datasets from file
with open(europarl_english_path, 'r') as corpus:
    english_corpus = corpus.read().lower()
    
# europarl_spanish_path = "dataset/Europarl.en-es.es"
europarl_spanish_path = "dataset/europarl-v7.es-en.es"
with open(europarl_spanish_path, 'r') as corpus:
    spanish_corpus = corpus.read().lower()

# Split corpi into sentences for later training and inference.
english = english_corpus.split('\n')
spanish = spanish_corpus.split('\n')

assert len(english) == len(spanish), f"Number of sentences between both languages is not equal! {len(english):,} vs {len(spanish):,}"
total_sentences = len(english)
    
# Some basic statistics about the corpi
print(f'Length in Words: EN: {len(english_corpus):,} | ES: {len(spanish_corpus):,} | Difference: {abs(len(english_corpus) - len(spanish_corpus)):,}')
print(f'Number of Sentences: EN: {len(english):,} | ES: {len(spanish):,}')

# Show an example sentence trainslation
import random

ran_sen = random.randint(0, total_sentences)
sample_english = english[ran_sen]
sample_spanish = spanish[ran_sen]

print(f'Sample Translation:\n')
print(f'\t{sample_english}\n')
print(f'\t{sample_spanish}')


Length in Words: EN: 294,629,416 | ES: 318,950,453 | Difference: 24,321,037
Number of Sentences: EN: 1,965,735 | ES: 1,965,735
Sample Translation:

	another positive aspect is the possibility for each member state to define protection areas and risk areas.

	otro aspecto positivo es la posibilidad de que cada estado miembro defina sus áreas de protección y áreas de riesgo.


## Tokenizing the Sentences and Collect Vocabulary

Tokenization is helpful for spliting the words and the special characters to feed into the model so that it can process it. In out case, we split between spaces and special characters. For example, the sentence `Today's dollar.` would be split into -> [`today`, `'`, `s`, `dollar`, `.`].

Further, the vocabulary is obtained for each language. This vocabulary will contain the `MAX_VOCABULARY` most populat words in each corpus.

In [10]:
special_chars = ',?;.:/*!+-()[]{}"\'&'

print('Working on English...')
english_sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in english ]
# Removes tokens that were empty. This helps in separating special characters!
english_tokens = [[w for w in s if len(w)] for s in english_sentences]

# English vocabulary calculation
english_words = [w for s in english_tokens for w in s]

english_vocab = Counter(english_words).most_common(MAX_VOCABULARY)
english_vocab = [w[0] for w in english_vocab]

print('Working on Spanish...')
spanish_sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in spanish ]
# Removes tokens that were empty. This helps in separating special characters!
spanish_tokens = [[w for w in s if len(w)] for s in spanish_sentences]

# Spanish vocabulary calculation
spanish_words = [w for s in spanish_tokens for w in s]

spanish_vocab = Counter(spanish_words).most_common(MAX_VOCABULARY)
spanish_vocab = [w[0] for w in spanish_vocab]

Working on English...
Working on Spanish...


## Save Languages

This is helpful for loading the languages later, skipping over all of the above processing from the raw files.

The sentences are stored in a dictionary, separating among actual tokenized instances, vocabulary, and a flag over the language. Numerical information such as the number of sentences or MAX_VOCABULARY can be extracted from the data itself, so it is not included.

In [11]:
import pickle

save_languages = True

english_europarl = {
    'tokens':     english_tokens,
    'vocabulary': english_vocab,
    'language': 'en',
}

spanish_europarl = {
    'tokens':     spanish_tokens,
    'vocabulary': spanish_vocab,
    'language': 'es',
}

if save_languages:
    
    with open('dataset/Europarl.en.pkl', 'wb') as jar:
        pickle.dump(english_europarl, jar)
        
    with open('dataset/Europarl.es.pkl', 'wb') as jar:
        pickle.dump(spanish_europarl, jar)

In [12]:
english_europarl['vocabulary']

['the', ',', '.', 'of', 'to', 'and', 'in', 'that', 'a', 'is']